In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from polarDensity_helper import Coord_Get, get_header_info
from site_distributions import make_simple_site, combine_sites, plot_density, make_symmetric_sites

In [ ]:
root = Path("/home/ems363/Projects/ELIC_PE_PG/aggregated")
replicas = ["liam/N50", "rep1/N50", "rep3"]
lipid = "POPG"

In [ ]:
leaf = "upp"
MB =11
AB = 0
EB = 2
fig, axes = plt.subplots(3,5, figsize=(15,10), sharex=True, sharey=True)
axis = 0
p_occ = pd.DataFrame(columns=["AB", "MB", "EB"], index=np.arange(0,14))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    try:
        cheng_sites = make_symmetric_sites(the_data, 4, 3, 24, 30, Ntheta, dr, dth, exrho, frames, sitename="Cheng Site ")
    except:
        print(f"Error on rep {rep}")
    #cheng_site_symmetric = combine_sites(cheng_sites, exrho, "Cheng Sites, Symmetric", symmetric=True)
    #cheng_sites.append(cheng_site_symmetric)
    for site in cheng_sites:
        site.Npeak = 0
        site.expPunocc = 0.302

    for site, ax, idx in zip(cheng_sites, axes.flatten()[axis:], np.arange(0,5)):
        p_occ.at[idx+axis, "AB"] = np.sum(site.densities[AB+1:])
        p_occ.at[idx+axis, "MB"] = np.sum(site.densities[MB+1:])
        p_occ.at[idx+axis, "EB"] = np.sum(site.densities[EB+1:])
        ax = plot_density(site, ax)

    axis+=len(cheng_sites)
 
fig.tight_layout()
plt.savefig(data_root.joinpath(f"Cheng_sites.pdf"))

In [ ]:
np.average(p_occ, axis=0)

In [ ]:
np.std(p_occ, axis=0)/np.sqrt(15)

In [ ]:
leaf = "upp"
lipid="POPG"
fig, ax = plt.subplots(1,1, figsize=(5,5))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    cheng_sites = make_symmetric_sites(the_data, 4, 3, 24, 30, Ntheta, dr, dth, exrho, frames, sitename="Cheng Site ")
    #cheng_site_symmetric = combine_sites(cheng_sites, exrho, "Cheng Sites, Symmetric", symmetric=True)
    #cheng_sites.append(cheng_site_symmetric)
    for site in cheng_sites:
        site.Npeak = 0
        site.expPunocc = 0.302

    dummy_site = cheng_sites[0]
    dummy_site.title=f"average over sites, {lipid}, {leaf}er leaflet, Cheng site"
    temp = [site.densities for site in cheng_sites]
    densities = [np.pad(dens, (0,18-len(dens))) for dens in temp]
    dummy_site.densities = np.average(densities, axis=0)

    ax = plot_density(dummy_site, ax)

plt.savefig(root.joinpath(f"Cheng_sites_averaged_{lipid}.pdf"))

In [ ]:
leaf = "low"
MB =11
AB = 0
EB = 0
fig, axes = plt.subplots(3,5, figsize=(15,10), sharex=True, sharey=True)
axis = 0
p_occ = pd.DataFrame(columns=["AB", "MB", "EB"], index=np.arange(0,14))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    try:
        ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
    except:
        print(f"Error on rep {rep}")
    #ulens_sites.append(ulens_site_symmetric)
    for site in ulens_sites:
        site.Npeak=0
        site.expPunocc = 0.187

    for site, ax, idx in zip(ulens_sites, axes.flatten()[axis:], np.arange(0,5)):
        p_occ.at[idx+axis, "AB"] = np.sum(site.densities[AB+1:])
        p_occ.at[idx+axis, "MB"] = np.sum(site.densities[MB+1:])
        p_occ.at[idx+axis, "EB"] = np.sum(site.densities[EB+1:])
        ax = plot_density(site, ax)

    axis+=len(cheng_sites)
 
fig.tight_layout()
plt.savefig(data_root.joinpath(f"Ulens_sites.pdf"))

In [ ]:
np.average(p_occ, axis=0)

In [ ]:
np.std(p_occ, axis=0)/np.sqrt(15)

In [ ]:
# leaf = "low"
# lipid = "POPG"
# for rep in replicas:
#     data_root = root.joinpath(rep)
#     fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

#     num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
#     the_data = np.loadtxt(fpath, skiprows=1)
#     rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

#     ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
#     #ulens_site_symmetric = combine_sites(ulens_sites, exrho, "Ulens Sites, Symmetric", symmetric=True)
#     #ulens_sites.append(ulens_site_symmetric)
#     for site in ulens_sites:
#         site.Npeak=0
#         site.expPunocc = 0.187

#     fig, axes = plt.subplots(3,2, figsize=(15,15))
#     for site, ax in zip(ulens_sites, axes.flatten()):
#         ax = plot_density(site, ax)

#     plt.savefig(data_root.joinpath(f"Ulens_sites.pdf"))

In [ ]:
# leaf = "low"
# for rep in replicas:
#     data_root = root.joinpath(rep)
#     fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

#     num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
#     the_data = np.loadtxt(fpath, skiprows=1)
#     rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

#     ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
#     ulens_sites.append(ulens_site_symmetric)
#     for site in ulens_sites:
#         site.Npeak=0
#         site.expPunocc = 0.187

#     dummy_site = ulens_sites[0]
#     dummy_site.title="average over sites"
#     temp = [site.densities for site in ulens_sites]
#     densities = [np.pad(dens, (0,50-len(dens))) for dens in temp]
#     dummy_site.densities = np.average(densities, axis=0)

#     fig, ax = plt.subplots(1,1, figsize=(10,10))
#     ax = plot_density(dummy_site, ax)

#     plt.savefig(data_root.joinpath(f"Ulens_sites_averaged.pdf"))

In [ ]:
leaf = "low"
lipid = "POPG"
fig, ax = plt.subplots(1,1, figsize=(5,5))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
    #ulens_sites.append(ulens_site_symmetric)
    for site in ulens_sites:
        site.Npeak=0
        site.expPunocc = 0.187

    dummy_site = ulens_sites[0]
    dummy_site.title="average over sites"
    temp = [site.densities for site in ulens_sites]
    densities = [np.pad(dens, (0,30-len(dens))) for dens in temp]
    dummy_site.densities = np.average(densities, axis=0)

    ax = plot_density(dummy_site, ax)

plt.savefig(root.joinpath(f"Ulens_sites_averaged_{lipid}.pdf"))